<a href="https://colab.research.google.com/github/chandutr/Deep-Learning-Projects/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
corpus=[
    "I like to eat pizza",
    "I like to eat pasta",
    "I like to eat Biriyani",
    "I love to eat Burger",
    "I love to play cricket"
]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)

In [ ]:
tokenizer.texts_to_sequences(["I like to eat pizza"])

[[1, 4, 2, 3, 6]]

In [ ]:
sequences=[]
#convert each sentence into input-output pairs
for line in corpus:
  tokenized=tokenizer.texts_to_sequences([line])[0]
  for i in range(2,len(tokenized)+1):
    seq=tokenized[:i]
    sequences.append(seq)
sequences

[[1, 4],
 [1, 4, 2],
 [1, 4, 2, 3],
 [1, 4, 2, 3, 6],
 [1, 4],
 [1, 4, 2],
 [1, 4, 2, 3],
 [1, 4, 2, 3, 7],
 [1, 4],
 [1, 4, 2],
 [1, 4, 2, 3],
 [1, 4, 2, 3, 8],
 [1, 5],
 [1, 5, 2],
 [1, 5, 2, 3],
 [1, 5, 2, 3, 9],
 [1, 5],
 [1, 5, 2],
 [1, 5, 2, 10],
 [1, 5, 2, 10, 11]]

In [ ]:
#pad sequences to have the same length
max_seq_len=max(len(seq) for seq in sequences)
sequences=pad_sequences(sequences, maxlen=max_seq_len,padding='pre')
sequences[:5]

array([[0, 0, 0, 1, 4],
       [0, 0, 1, 4, 2],
       [0, 1, 4, 2, 3],
       [1, 4, 2, 3, 6],
       [0, 0, 0, 1, 4]], dtype=int32)

In [ ]:
max_seq_len

5

In [ ]:
sequences=np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
X[:5], y[:5]

(array([[0, 0, 0, 1],
        [0, 0, 1, 4],
        [0, 1, 4, 2],
        [1, 4, 2, 3],
        [0, 0, 0, 1]], dtype=int32),
 array([4, 2, 3, 6, 4], dtype=int32))

In [ ]:
#one hot encode the labels
vocab_size=len(tokenizer.word_index)
y=to_categorical(y,num_classes=vocab_size+1)
y[:5]

array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
vocab_size

11

In [ ]:
#build rnn model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,Dense

model=Sequential([
    Embedding(input_dim=vocab_size+1, output_dim=10, input_length=max_seq_len-1),
    SimpleRNN(50),
    Dense(vocab_size+1,activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X,y,epochs=200)

Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0500 - loss: 2.4818
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.3500 - loss: 2.4650
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.3000 - loss: 2.4479
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.4500 - loss: 2.4304
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.4500 - loss: 2.4123
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4500 - loss: 2.3934
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.4500 - loss: 2.3734
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.4500 - loss: 2.3524
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 0.4500 - loss: 2.3299
Epoch 10/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.4500 - loss: 2.3060
Epoch 11/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.4500 - loss: 2.2805
Epoch 12/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4500 - loss

In [ ]:
#prediction function
def predict_next_word(model,tokenizer,text,max_seq_len):
  tokens=tokenizer.texts_to_sequences([text])[0]
  tokens=pad_sequences([tokens], maxlen=max_seq_len-1,padding='pre')
  predicted=model.predict(tokens)
  next_index=np.argmax(predicted)
  for word,index in tokenizer.word_index.items():
    if index==next_index:
      return word
  return ""

In [ ]:
tokenizer.word_index.items()

dict_items([('i', 1), ('to', 2), ('eat', 3), ('like', 4), ('love', 5), ('pizza', 6), ('pasta', 7), ('biriyani', 8), ('burger', 9), ('play', 10), ('cricket', 11)])

In [ ]:
#test the model
test_input="I like to eat"
predicted_word=predict_next_word(model,tokenizer,test_input,max_seq_len)
print(f"{test_input} -> {predicted_word}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
I like to eat -> pizza
